# Introduction to cheminformatics

Andrea Volkamer - adapted by Gautier Peyrat

## Learning objectives

### Basic handling of molecules

* Reading & writing of molecules
* Molecular descriptors & fingerprints
* Molecular similarity

### Using RDKit: open source cheminformatics software

More information can be found here:

* http://www.rdkit.org/docs/index.html
* http://www.rdkit.org/docs/api/index.html

In [ ]:
# The majority of the basic molecular functionality is found in module rdkit.Chem
from rdkit import Chem
from rdkit.Chem import AllChem

## Representation of molecules

You have perhaps encountered many ways of representing chemicals, and here we will list a few:

- Trivial Names (Aspirin)
- Systematic Names (2-acetyloxybenzoic acid)
- Formula (C<sub>9</sub>H<sub>8</sub>O<sub>4</sub>)
- Images (https://en.wikipedia.org/wiki/File:Aspirin-B-3D-balls.png)

But how do **chemists** communicate with **computers**?

2 main methods:
- molecular graph (connection tables)
- line notation

We will focus on the **line notation**, because:

1. many computational processes operate more effectively on data structured as linear strings than data structured as tables.
2. line notations can be reasonably legible to human chemists designing functions with these tools.

One of the most frequently used line notation in chemistry is:

### SMILES (Simplified Molecular Input Line Entry Specification)

* Atoms are represented by atomic symbols: C, N, O, F, S, Cl, Br, I
* Double bonds are `=`, triple bonds are `#`
* Branching is indicated by parenthesis
* Ring closures are indicated by pairs of matching digits

More information can be found here: http://www.daylight.com/dayhtml/doc/theory/theory.smiles.html

In [ ]:
# Individual molecules can be constructed using a variety of approaches
# FDA approved EGFR inhibitors: Gefitinib, Erlotinib

mol1 = Chem.MolFromSmiles('COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1')
mol2 = Chem.MolFromSmiles('C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1')

Above cell creates what we often called "RDKit molecule object" from their SMILES.  
We can check by using the `type` command:

In [ ]:
type(mol1)

#### Drawing molecules

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [ ]:
# Single molecule
mol1

In [ ]:
# List of molecules
Draw.MolsToGridImage([mol1,mol2], useSVG=True)

In [ ]:
# A single molecule has different SMILES 
mol3 = Chem.MolFromSmiles('C1=CC=CN=C1')
mol4 = Chem.MolFromSmiles('c1cccnc1')
mol5 = Chem.MolFromSmiles('n1ccccc1')
Chem.Draw.MolsToGridImage([mol3, mol4, mol5])

In [ ]:
#By default RDKit returns the canonical SMILES
print(Chem.MolToSmiles(mol3))
print(Chem.MolToSmiles(mol4))
print(Chem.MolToSmiles(mol5))

#### Stereochemistry

[Wikpedia](https://en.wikipedia.org/wiki/Stereochemistry)  
The famous image of different types of isomers:
![The famous image of different types of isomers](https://upload.wikimedia.org/wikipedia/commons/0/04/Isomerism.svg)

##### Cis–trans isomerism

Example: Geraniol and Nerol

https://en.wikipedia.org/wiki/Geraniol  
https://en.wikipedia.org/wiki/Nerol

https://pubs.acs.org/doi/10.1021/acs.jafc.6b04534

    Geraniol, or (2E)-3,7-dimethylocta-2,6-dien-1-ol, and its Z-isomer, nerol, are fragrant substances of great value.  
    The smell of geraniol has previously been described as sweet, fruity, and berry-like, whereas nerol has been reported as having floral, citrus-like, and sweet-smelling properties. 

Encoded in the SMILES with `/` and `\` symbols

In [ ]:
mol6 = Chem.MolFromSmiles("O/C=C/Cl") # E or trans isomer
mol6

In [ ]:
mol7 = Chem.MolFromSmiles('O/C=C\Cl') # Z or cis isomer
mol7

##### Chirality

Example: https://en.wikipedia.org/wiki/Thalidomide

Chirality is encoded with `@` or `@@` symbol

In [ ]:
mol8 = Chem.MolFromSmiles('Oc1ccc(cc1)/C=C/c1cc(O)cc2c1C(c1cc(O)cc(c1)O)C(O2)c1ccc(cc1)O')
mol8

In [ ]:
mol9 = Chem.MolFromSmiles('Oc1ccc(cc1)/C=C/c1cc(O)cc2c1[C@@H](c1cc(O)cc(c1)O)[C@@H](O2)c1ccc(cc1)O')
mol9

#### other representations: InChI/InChIKey/SDF

In [ ]:
# Inchi
print(Chem.MolToInchi(mol1))

In [ ]:
# InchiKey
print(Chem.MolToInchiKey(mol1))

In [ ]:
# Inchikeys from two molecules with different stereochemistry

print(Chem.MolToInchiKey(mol8))
print(Chem.MolToInchiKey(mol9))

In [ ]:
#Si on a une molécule neutre et pas neutre :
mol10 = Chem.MolFromSmiles('[H]N1CCCC1C(O)=O')
mol11 = Chem.MolFromSmiles('[H]N1CCCC1C([O-])=O')
mol12 = Chem.MolFromSmiles('[NH2+]1CCCC1C([O-])=O')
mol13 = Chem.MolFromSmiles('[NH2+]1CCCC1C(O)=O')

Chem.Draw.MolsToGridImage([mol10, mol11, mol12, mol13], molsPerRow=4)

Question for biologists and chemists:
- what is the name of the molecule that you saw in above output cell?
- why it has 4 different charged/uncharges forms?
- what is the professional term for this?
- what is the factor that will influence its form?

In [ ]:
for mol in [mol10, mol11, mol12, mol13] :
    print(Chem.MolToInchiKey(mol))

In [ ]:
# MolBlock
print(Chem.MolToMolBlock(mol1))

For more details about the definition of MolBlock of an SDF file:

https://chem.libretexts.org/Courses/Intercollegiate_Courses/Cheminformatics/02%3A_Representing_Small_Molecules_on_Computers/2.05%3A_Structural_Data_Files

### Generating 3D coordinates

In [ ]:
m_3D = Chem.AddHs(mol1)
AllChem.EmbedMolecule(m_3D)
#AllChem.UFFOptimizeMolecule(m_3D) # Improves the quality of the conformation; this step should not be necessary since v2018.09: default conformations use ETKDG
Draw.MolsToGridImage([mol1,m_3D])

In [ ]:
print(Chem.MolToMolBlock(m_3D))

### Writing molecules to *sdf* (structure data files)

In [ ]:
w = Chem.SDWriter('./data/mytest_mol3D.sdf')
w.write(m_3D)
w.close()

### Get information on molecules

In [ ]:
mol2

Number of heavy atoms (C, O, N, F, Cl ...), not H

In [ ]:
mol2.GetNumHeavyAtoms()

Number of bonds

In [ ]:
mol2.GetNumBonds()

### Pandas Dataframe

`Pandas` is a fast, powerful, flexible and easy to use open source data analysis and manipulation Python tool.
It allows to:
 - Manipulate data tables with labels for variables (columns) and individuals (rows).
 - These tables are called DataFrames, similar to dataframes under R.
 - Read and write these dataframes from or to a tabulated file.
 - Plot graphs from these DataFrames with `matplotlib` (or other similar packages).

Here we combine Pandas dataframes and RDKit to display molecules in tables.

In [ ]:
import pandas as pd   # "as pd" creates an alias to simplify the call of pandas library
from rdkit.Chem.PandasTools import RenderImagesInAllDataFrames
RenderImagesInAllDataFrames(images=True)

Exercise

In [ ]:
# Get the smiles of 3 drugs (Aspirin, Paracetamol and Ibuprofen) and convert them to RDKit molecules

# Create a list named "Lmeds" containing the molecule objects of these three drugs

Create a dataframe with the information in above list:

In [ ]:
df_meds=pd.DataFrame()
df_meds['ID'] = ['Aspirin', 'Paracetamol', 'Ibuprofen']
df_meds['Molecule'] = Lmeds
df_meds

In [ ]:
# Add heavy atom number to each row of the dataframe.
# Firstly, create an empty list called L_heavyatoms

# Then append the number of heavy atoms to this list, by using RDKit molecule GetNumHeavyAtoms() method

Add new column to the dataframe

In [ ]:
df_meds['Heavy Atoms'] = L_heavyatoms 
df_meds

Another way to add a new column is to use "apply" directly on the dataframe :

In [ ]:
from rdkit.Chem import Fragments

In [ ]:
# Apply avoids to do a for loop and store the result in a list to append a new column in the dataframe
df_meds['Carboxylic_acid'] = df_meds.Molecule.apply(Fragments.fr_COO)
df_meds

# Quiz

In [ ]:
from nbautoeval import run_yaml_quiz

In [ ]:
run_yaml_quiz(f"../corrections/quiz/intro.yaml", "theoric-quiz_cheminf")

In [ ]:
run_yaml_quiz(f"../corrections/quiz/intro.yaml", "code-quiz_cheminf")